In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

spray = pd.read_csv("../dsi5/project-4/data/spray.csv").drop_duplicates().dropna()
train = pd.read_csv("../dsi5/project-4/data/train.csv").drop_duplicates().dropna()
mapdata = np.loadtxt("../dsi5/project-4/data/mapdata_copyright_openstreetmap_contributors.txt")

In [59]:
spray.drop(columns=["Time"], inplace=True)
spray.Date = pd.to_datetime(spray.Date)
spray.set_index('Date',inplace=True)

In [62]:
mosquitos = train[['Longitude', 'Latitude','NumMosquitos','Date']];
mosquitos.Date = pd.to_datetime(mosquitos.Date);
mosquitos_counts = mosquitos.groupby('Date')['NumMosquitos'].sum();
mosquitos.set_index("Date", inplace=True);
mosquitos["Counts"] = mosquitos_counts

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [63]:
west_nile = train[train['WnvPresent'] ==1 ][['Longitude', 'Latitude',"Date","WnvPresent"]]
west_nile.Date = pd.to_datetime(west_nile.Date)
west_nile_counts = west_nile.groupby(['Date'])['WnvPresent'].sum()
west_nile.set_index("Date", inplace=True)
west_nile["Counts"] = west_nile_counts

In [64]:
# Weather
# fig, axarr = plt.subplots(nrows = 1, ncols = len(models), figsize=(24, 7))

In [65]:
dates = list(map(lambda x: str(x).replace(" 00:00:00",""), sorted(list(set(list(spray.index) + list(mosquitos.index) + list(west_nile.index))))))

In [ ]:
# Need to create unique years, months, days dates
# to loop through like a small clock
years, months, days = [sorted(list(set(map(lambda x: x.split("-")[i], dates)))) for i in range(0,3)]

ims = []

fig = plt.figure(figsize=(10,14))

for year in years:
    for month in months[3:6]:
        for day in days[8:12]:
            # Skip "31" for June and September
            # Otherwise we will be out of bounds
            if day == "31" and (month == "06" or month == "09"):
                continue
            # I could have done these in separate loops
            # But I was afraid that would create three separate animations
            # So the code has a lot of duplication, sorry...
            spray_subset = spray[spray.index <=f"{year}-{month}-{day}"]
            spray_mask = (spray_subset['Longitude'] >= -88.0)
            spray_locations = spray_subset[spray_mask][['Longitude', 'Latitude']].dropna().values
            
            mosquitos_subset = mosquitos[mosquitos.index <=f"{year}-{month}-{day}"]
            mosquitos_locations = mosquitos_subset[['Longitude', 'Latitude','NumMosquitos']].dropna().values
            
            west_nile_subset = west_nile[west_nile.index <=f"{year}-{month}-{day}"]
            west_nile_locations = west_nile_subset[['Longitude', 'Latitude','WnvPresent']].dropna().values
        
            aspect = mapdata.shape[0] * 1.0 / mapdata.shape[1]
            lon_lat_box = (-88, -87.5, 41.6, 42.1)
            plt.imshow(mapdata, 
                cmap=plt.get_cmap('gray'), 
                extent=lon_lat_box, 
                aspect=aspect,
                animated=True
            )
            plt.scatter(spray_locations[:,0], spray_locations[:,1], marker='x', color="blue")
            plt.scatter(mosquitos_locations[:,0], mosquitos_locations[:,1], marker='o', color="orange", s=mosquitos_locations[:,2]**2)
            ims.append([plt.scatter(west_nile_locations[:,0], west_nile_locations[:,1], marker='*', color="red", s=west_nile_locations[:,2]*10)])

ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True)

ani.save('./spray_mosquitos_wnv.mp4')

HTML(ani.to_html5_video())